# Day 2 - (Agent Tools & Interoperability with Model Context Protocol (MCP))

> ⚠️ **Nota importante**  
>  A continuación se muestra un proceso secuencial inicial, a nivel de configuración, que es necesario realizar antes de empezar a trabajar con los Agentes. Se importan librerias de ADK necesarias con sus respectivos componentes, se crean las Helper Functions y las opciones de configuración de Retry.  


### Import ADK components. ###

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [2]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### Helper Functions ###

Helper function that prints the generated Python code and results from the code execution tool:

In [3]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


### Configure Retry Options ###

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

---

> ⚠️ **Nota importante**  
> Empieza aquí realmente el trabajo de estudio, finalizan aquí las configuraciones.
> 

# Custom tools ###

#### 🛠️ Buenas prácticas para crear nuestras propias tools en Python

- La función devuelve un diccionario (`{}`).
- Crear *docstrings* claras y limpias.
- Manejar los errores.
- Especificar el tipo de dato en las funciones.  
  Ejemplo:

  ```python
  def get_time(lugar: str) -> dict:
      ...


### Custom Tool: Agente conversor de moneda.

Creamos nuestra primera Custom Tool. Puedes poner atención en el return (dict), en el docstring claro, manejo de errores y el type hint (especificar el tipo de variable, no necesario en programación tradicional pero en este caso, sirve de contexto al LLM:

In [10]:
def get_fee_for_payment_method(method: str) -> dict:

    """

    Busca el porcentaje de comisión de transacción para un método de pago determinado.

    Esta herramienta simula la consulta de la estructura interna de comisiones de una empresa
    basándose en el nombre del método de pago proporcionado por el usuario.

    Args:
    
        method: El nombre del método de pago. Debe ser descriptivo,
                por ejemplo, "tarjeta de crédito platinum" o "transferencia bancaria".

    Returns:
    
        Diccionario con el estado y la información de la comisión.
        Éxito: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Método de pago no encontrado"}

    """

    # Esto simula la consulta de la estructura interna de comisiones de una empresa.
    fee_database = {
        
        "tarjeta platinum": 0.02,               # 2%
        "visa Oro": 0.035,                      # 3.5%
        "transferencia bancaria": 0.01,         # 1%
        
        }

    fee = fee_database.get(method.lower())

    if fee is not None:
        return {"status" : "success", "fee_percentage" : fee}

    else:
        return {
            "status" : "error",
            "error_message" : f"Método de pago {method} no encontrado"

        }

print("✅ Fee lookup function created")
print(f"💳 Test: {get_fee_for_payment_method('tarjeta Platinum')}")

✅ Fee lookup function created
💳 Test: {'status': 'success', 'fee_percentage': 0.02}


A continuación, creamos otra función para obtener la tasa de conversión entre monedas, y usar la función como tool para el Agente IA.

In [19]:
def get_exchange_rate(base_currency : str, target_currency: str) -> dict :

    """

    Busca y devuelve el tipo de cambio entre dos monedas.

    Args:
    
        base_currency: El código de moneda ISO 4217 de la moneda
                       desde la cual se realiza la conversión
                       (por ejemplo, "USD").
        target_currency: El código de moneda ISO 4217 de la moneda
                         a la cual se realiza la conversión
                         (por ejemplo, "EUR").

    Returns:
    
        Diccionario con el estado y la información del tipo de cambio.
        Éxito: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Par de monedas no compatible"}

    """

    # A continuación, datos estáticos que simulan una API de tipos de cambio en tiempo real
    # En producción, esto llamaría a algo como: requests.get("api.exchangerates.com")

    rate_database = {
        
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

        
    # Validación y procesamiento de la entrada. Todo a minusuclas, cero problemas.
    base = base_currency.lower()
    target = target_currency.lower()

    #Return estructurado y con "status":
    rate = rate_database.get(base,{}).get(target)

    if rate is not None:
        return {"status" : "success", "rate" : rate}

    else:
        return {
            
            "status" : "error", 
            "error_message": f"Par de moendas no soportado: {base_currency} / {target_currency}"

        }


print("✅ Exchange rate function created")
print(f"💱 Test: {get_exchange_rate('USD', 'EUR')}")        

    
        

✅ Exchange rate function created
💱 Test: {'status': 'success', 'rate': 0.93}


Ahora vamos a crear a nuestro Agente IA conversor de monedas. Si pones atención, verás como las "instruction" del Agente hacen referencia a las "tools".

Puntos Clave:

- La lista ***tools = []*** le indica al Agente qué funciones puede utilizar.
- Las ***instruction*** hacen referencia a las ***tools*** usando exactamente los nombres de las funciones(ej. *get_fee_for_payment_method()*
- El Agente utiliza estos nombres para decidir cúando y cómo llamar a cada ***tool***

In [28]:
currency_agent = LlmAgent(
    
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_optionn=retry_config),
    
    instruction="""

    Eres un asistente inteligente de conversión de monedas.

    Para solicitudes de conversión de divisas:
    
    1. Usa get_fee_for_payment_method() para obtener las comisiones de la transacción.
    2. Usa get_exchange_rate() para obtener los tipos de cambio.
    3. Verifica el campo 'status' en la respuesta de cada tool para detectar errores.
    4. Calcula el monto final después de aplicar las comisiones basándote en los resultados de
       get_fee_for_payment_method y get_exchange_rate, y proporciona un desglose claro.
    5. Primero, indica el monto final convertido.
       Luego, explica cómo llegaste a ese resultado mostrando los valores intermedios.
       Tu explicación debe incluir: el porcentaje de la comisión y su valor en la moneda original,
       el monto restante después de aplicar la comisión, y el tipo de cambio usado para la conversión final.
    
    Si alguna tool devuelve un estado 'error', explica claramente el problema al usuario.

    
                    """,

    tools = [get_fee_for_payment_method, get_exchange_rate]


)

print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")
    
    

✅ Currency agent created with custom function tools
🔧 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


Procedemos ahora a probar el Agente:

In [32]:
currency_runner = InMemoryRunner(agent=currency_agent)
                                 
_ = await currency_runner.run_debug(
    "Quiero convertir 500 Dólares Americanos a Euros usando la forma de transferencia bancaria. ¿Cuántos Euros recibiré?"
)

    


 ### Created new session: debug_session_id

User > Quiero convertir 500 Dólares Americanos a Euros usando la forma de transferencia bancaria. ¿Cuántos Euros recibiré?
currency_agent > Analicemos la conversión de 500 Dólares Americanos (USD) a Euros (EUR) usando transferencia bancaria.

Primero, la comisión por transferencia bancaria es del 1% sobre el monto original.
Comisión = 500 USD * 0.01 = 5 USD.

El monto después de aplicar la comisión es:
Monto restante = 500 USD - 5 USD = 495 USD.

Ahora, aplicamos el tipo de cambio de USD a EUR, que es 0.93.
Euros recibidos = 495 USD * 0.93 = 460.35 EUR.

Por lo tanto, recibirás 460.35 EUR.

Desglose:
- Comisión de transferencia bancaria: 1% (5 USD)
- Monto a convertir después de la comisión: 495 USD
- Tipo de cambio: 1 USD = 0.93 EUR
- Monto final en EUR: 460.35 EUR


## Mejorando la fiabilidad del agente usando código ##

Las instrucciones del agente dicen “calcula el monto final después de las comisiones”, pero ***los LLM no siempre son fiables haciendo matemáticas***. Pueden cometer errores de cálculo o usar fórmulas inconsistentes.

💡 Solución: vamos a pedirle a nuestro agente que genere código en Python para realizar los cálculos y lo ejecutaremos para obtener el resultado final.
¡La ejecución de código es mucho más fiable que intentar que el LLM haga las cuentas “de cabeza”!

*Ejecutor de Código Integrado*

ADK incluye un Ejecutor de Código integrado capaz de ejecutar código en un entorno aislado (sandbox).
Nota: esto utiliza la capacidad de Ejecución de Código de Gemini.

***Ahora vamos a crear un calculation_agent que reciba código en Python y use el BuiltInCodeExecutor para ejecutarlo***.

In [34]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""


                Eres una calculadora especializada que SOLO responde con código Python.
                Tienes prohibido proporcionar cualquier texto, explicaciones o respuestas conversacionales.
                
                Tu tarea es tomar una solicitud de cálculo y traducirla en un único bloque de código Python
                que calcule la respuesta.
                
                **REGLAS:**
                1. Tu salida DEBE ser SOLO un bloque de código Python.
                2. NO escribas ningún texto antes ni después del bloque de código.
                3. El código Python DEBE calcular el resultado.
                4. El código Python DEBE imprimir el resultado final en stdout.
                5. TIENES PROHIBIDO realizar el cálculo tú mismo. Tu único trabajo es generar el código que realizará el cálculo.
                
                No cumplir estas reglas dará lugar a un error.



                """,

    code_executor = BuiltInCodeExecutor(),  # Usa el Ejecutor de Código integrado. Esto le da al agente capacidades de ejecución de código
)

    

***Ahora vamos actualizar las instrucciones y el conjunto de tools del agente.***

Vamos a realizar dos acciones clave:

- Actualizar las instrucciones del currency_agent para que genere código Python

    Original:
    “Calcular el monto final después de las comisiones” (instrucciones matemáticas vagas)
    
    Mejorado:
    “Genera código Python para calcular el monto final … y usa el calculation_agent para ejecutar el código y obtener el monto final”

- Añadir el calculation_agent al conjunto de tools

ADK permite usar cualquier agente como una tool mediante AgentTool.

Añade *AgentTool(agent=calculation_agent)* a la lista tools. (El agente especialista aparece como una tool invocable para el agente principal)



In [38]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Instrucciones actualizadas
    instruction="""
    
    Eres un asistente inteligente de conversión de monedas. Debes seguir estrictamente estos pasos y usar las tools disponibles.

    Para cualquier solicitud de conversión de divisas:
    
    1. Obtener comisión de transacción: Usa la tool get_fee_for_payment_method() para determinar la comisión de la transacción.
    
    2. Obtener tipo de cambio: Usa la tool get_exchange_rate() para obtener el tipo de cambio entre monedas.
    
    3. Verificación de errores: Después de cada llamada a una tool, debes comprobar el campo "status" en la respuesta. 
       Si el estado es "error", debes detenerte y explicar claramente el problema al usuario.
       
    4. Calcular el monto final (CRÍTICO): Tienes estrictamente prohibido realizar cálculos aritméticos por tu cuenta. 
       Debes usar la tool calculation_agent para generar código Python que calcule el monto final convertido. 
       Este código utilizará la información de la comisión del paso 1 y el tipo de cambio del paso 2.
       
    5. Proporcionar un desglose detallado: En tu resumen, debes:
       * Indicar el monto final convertido.
       * Explicar cómo se calculó el resultado, incluyendo:
           * El porcentaje de la comisión y el valor de la comisión en la moneda original.
           * El monto restante después de descontar la comisión.
           * El tipo de cambio aplicado.
           
                """,
    
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Usando otro agente como una tool
    ],
)

print("✅ Agente de conversión mejorado creado")
print("🎯 Nueva capacidad: delega los cálculos a un agente especialista")
print("🔧 Tipos de tools usadas:")
print("  • Function Tools (comisiones, tipos de cambio)")
print("  • Agent Tool (especialista en cálculos)")


✅ Agente de conversión mejorado creado
🎯 Nueva capacidad: delega los cálculos a un agente especialista
🔧 Tipos de tools usadas:
  • Function Tools (comisiones, tipos de cambio)
  • Agent Tool (especialista en cálculos)


In [40]:
# Define a runner
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

#### Ahora procedemos a probar el nuevo Agente mejorado (calculo por codigo Python)

In [42]:

response = await enhanced_runner.run_debug(
   
    "Convierte 1250 USD a INR usando una transferencia bancaria, muestrame el calculo preciso."
)


 ### Continue session: debug_session_id

User > Convierte 1250 USD a INR usando una transferencia bancaria, muestrame el calculo preciso.
enhanced_currency_agent > Aquí tienes el desglose detallado de tu conversión de 1250 USD a INR usando una transferencia bancaria:

*   **Monto inicial:** 1250 USD
*   **Tipo de cambio:** 1 USD = 83.58 INR
*   **Monto en INR antes de la comisión:** 104475.00 INR
*   **Tasa de comisión:** 1%
*   **Monto de la comisión:** 1044.75 INR
*   **Monto final convertido:** 103430.25 INR


***Excelente! Observa lo que ocurrió:***

Cuando el agente de conversión de moneda llama al CalculationAgent, le pasa el código Python generado.

El CalculationAgent, a su vez, utiliza el BuiltInCodeExecutor para ejecutar el código y nos proporciona cálculos precisos en lugar de conjeturas del LLM.

Ahora puedes inspeccionar las partes de la respuesta que generaron código Python o que contienen los resultados de la ejecución del código Python, usando la función auxiliar que se definió cerca del comienzo de este cuaderno (notebook).

In [43]:
show_python_code_and_result(response)

Generated Python Code >>  
usd_amount = 1250
exchange_rate = 83.58
commission_rate = 0.01

# Calculate the amount in INR without commission
inr_amount_before_commission = usd_amount * exchange_rate

# Calculate the commission amount
commission_amount = inr_amount_before_commission * commission_rate

# Calculate the final amount in INR after deducting commission
final_inr_amount = inr_amount_before_commission - commission_amount

print(f"Monto inicial en USD: {usd_amount}")
print(f"Tipo de cambio (1 USD = {exchange_rate} INR): {exchange_rate}")
print(f"Monto en INR antes de la comisión: {inr_amount_before_commission:.2f}")
print(f"Tasa de comisión: {commission_rate*100:.0f}%")
print(f"Monto de la comisión: {commission_amount:.2f}")
print(f"Monto final en INR: {final_inr_amount:.2f}")


